In [47]:
import pandas as pd
import numpy as np
import os
import glob

### Creamos el datamart final para entrenar el modelo

In [48]:
def load_homes_external(datalake_path):
    """
    Carga y une los archivos homes.tsv y external_indicators.tsv de todos los años en un único DataFrame.
    La unión se realiza por 'Año' y 'CCAA'.
    """
    all_homes = glob.glob(os.path.join(datalake_path, "*", "homes.tsv"))
    all_external = glob.glob(os.path.join(datalake_path, "*", "external_indicators.tsv"))

    df_list = []

    for home_file, external_file in zip(all_homes, all_external):
        year = os.path.basename(os.path.dirname(home_file))  # Extraer el año de la ruta

        df_homes = pd.read_csv(home_file, sep="\t")
        df_external = pd.read_csv(external_file, sep="\t")

        df_homes["Año"] = int(year)
        df_external["Año"] = int(year)

        # Unir por Año y CCAA
        df_merged = pd.merge(df_homes, df_external, on=["Año", "CCAA"], how="inner")
        df_list.append(df_merged)

    if not df_list:
        raise ValueError("No se encontraron archivos homes.tsv o external_indicators.tsv en la ruta del datalake.")

    # Concatenar todos los DataFrames
    df_homes_external = pd.concat(df_list, ignore_index=True)
    return df_homes_external

In [49]:
def merge_with_datamart(df_homes_external, datamart_path):
    """
    Une el DataFrame combinado de homes y external_indicators con el datamart de supercategorías.
    La unión se realiza por 'Año' y 'NUMERO'. Cada supercategoría se convierte en una columna
    con el gasto mensual correspondiente.
    """
    df_datamart = pd.read_csv(datamart_path, sep="\t")

    # Pivotar el datamart: cada SUPER_CATEGORIA como columna
    df_pivot = df_datamart.pivot_table(
        index=["Año", "NUMERO"],
        columns="SUPER_CATEGORIA",
        values="GASTOMON",
    ).reset_index()

    # Renombrar columnas para mayor claridad, por ejemplo: SC_011, SC_012, etc.
    df_pivot.columns = ['Año', 'NUMERO'] + [f"SC_{col}" for col in df_pivot.columns[2:]]

    # Unir con el DataFrame combinado
    df_final = pd.merge(df_homes_external, df_pivot, on=["Año", "NUMERO"], how="inner")
    return df_final


In [50]:
load_homes_external_path = "../../DataLake" 
datamart_path = "../../datamarts/datamart_supercategories.tsv"
output_path = "../../datamarts/final_dataframe.tsv" 

# Cargar y unir los archivos homes y external_indicators
df_homes_external = load_homes_external(load_homes_external_path)

# Unir con el datamart
df_final = merge_with_datamart(df_homes_external, datamart_path)


In [51]:
pd.set_option('display.max_columns', None)  
df_final.head(15)  # Mostrar las primeras filas del DataFrame final

,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NNINOSD,NHIJOSD,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTOT,GASTMON,GASTNOM4,FACTOR,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,NACION_EXT,EDUC_SUPERIOR,Año,Tasa_Paro,T_med,Inflacion,Tipo_Interes,EUR_USD,IPC,SC_1,SC_2,SC_3,SC_4,SC_5,SC_6,SC_7,SC_8,SC_9,SC_10,SC_11,SC_12
0,1,9,0,4,3,2.0,2.0,0,0,55,5,5,70.0,1,1,0,0.0,5020,autonomYRenta,4.260498e+09,3.669192e+09,5.913062e+08,9.026143e+08,2.0,58.0,0,0,1,1,0,0,1,0,1,0,1,0,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,6.849816e+08,319264308.0,3.428515e+07,175239084.0,5.039262e+08,210881486.0,3.140708e+08,210924601.0,5.338217e+08,18083827.0,1.077787e+07,6.529352e+08
1,4,9,1,5,3,1.0,1.0,1,1,49,4,5,65.0,1,1,0,0.0,1661,asalariado,1.157571e+09,1.157571e+09,NaN,4.608349e+08,1.0,36.0,0,1,0,1,0,0,0,0,1,1,1,0,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,3.065459e+08,8353353.0,1.357546e+08,161476554.0,2.554148e+07,80572815.0,2.317706e+07,52369740.0,1.070566e+08,66360228.0,9.435450e+07,9.600848e+07
2,5,9,0,1,1,1.0,1.0,2,2,38,5,1,120.0,1,1,0,0.0,1082,asalariado,1.935653e+09,1.532995e+09,4.026576e+08,8.357985e+08,1.0,72.0,1,1,1,1,0,0,1,1,0,2,1,0,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,2.071399e+08,117821010.0,2.885848e+08,103409729.0,5.370047e+07,13292166.0,1.127863e+08,39693246.0,2.730771e+08,29785347.0,1.784300e+08,1.152752e+08
3,11,9,0,1,2,0.0,0.0,0,0,74,6,3,70.0,1,1,0,0.0,1282,pension,2.506370e+09,2.106291e+09,4.000792e+08,9.048945e+08,2.0,60.0,0,0,0,0,1,1,1,0,1,0,1,0,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,5.257447e+08,44211243.0,1.351971e+08,221094635.0,1.472135e+07,57875692.0,7.005875e+08,106605537.0,1.030967e+08,NaN,2.257742e+07,1.745789e+08
4,34,9,0,4,3,0.0,0.0,0,0,48,6,5,56.0,1,1,0,0.0,1685,pension,2.311958e+09,1.361849e+09,9.501089e+08,1.718298e+09,1.0,23.0,0,0,1,0,0,0,1,0,0,0,1,0,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,2.995672e+08,70557541.0,NaN,293301126.0,NaN,NaN,NaN,189679497.0,1.816789e+08,NaN,3.005975e+08,2.646700e+07
5,64,9,0,1,2,2.0,2.0,1,1,47,5,7,120.0,1,1,0,0.0,3000,autonomYRenta,4.038602e+09,3.121573e+09,9.170283e+08,1.207538e+09,2.0,42.0,1,0,1,1,0,0,1,1,1,0,1,0,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,7.400974e+08,202145284.0,2.082520e+07,576793216.0,1.143687e+08,5983349.0,2.054432e+08,67672831.0,4.636648e+08,31396.0,2.476829e+08,4.768651e+08
6,75,9,1,4,3,0.0,0.0,0,0,74,5,5,75.0,1,1,0,0.0,788,pension,1.331639e+09,8.164914e+08,5.151479e+08,9.592000e+08,2.0,56.0,0,0,0,0,1,1,1,0,1,0,1,0,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,1.727784e+08,NaN,NaN,243840915.0,6.906997e+07,119726576.0,3.453787e+06,8632800.0,1.155357e+08,NaN,2.600804e+07,5.744512e+07
7,86,9,0,1,2,2.0,2.0,2,2,40,4,5,85.0,1,1,0,0.0,2286,asalariado,2.400327e+09,2.400327e+09,NaN,9.810996e+08,2.0,102.0,0,2,1,1,0,0,1,0,1,2,1,0,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,3.443574e+08,248759298.0,2.255485e+08,761725704.0,3.685010e+06,1745376.0,1.311602e+08,13578123.0,1.657325e+08,NaN,1.326396e+08,3.713952e+08
8,101,9,1,5,3,1.0,1.0,4,4,48,5,5,80.0,1,0,0,0.0,1600,asalariado,3.761038e+09,2.998535e+09,7.625025e+08,2.026596e+09,1.0,110.0,1,3,1,1,0,0,1,1,0,4,0,1,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,1.306698e+09,23733461.0,7.555356e+08,207732127.0,2.681702e+08,9383137.0,6.226512e+07,810638.0,3.693673e+07,NaN,6.687968e+07,2.603908e+08
9,117,9,0,1,2,2.0,2.0,2,2,38,6,6,300.0,1,1,0,0.0,8416,asalariado,2.202685e+10,1.922243e+10,2.361823e+09,1.188568e+09,2.0,104.0,0,2,1,1,0,0,1,0,1,2,1,0,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,8.537108e+08,283734568.0,2.137355e+09,908444509.0,3.941330e+09,219773992.0,6.271835e+09,18176941.0,1.380734e+09,157374984.0,5.760202e+08,2.473942e+09


In [52]:
# Eliminar columnas NHIJOSD e NNINOSD
df_final.drop(columns=["NHIJOSD", "NNINOSD"], inplace=True, errors='ignore')

In [53]:
# COntar el número de valores -9 por columna
pd.set_option('display.max_rows', None)  # Mostrar todas las filas
missing_counts = (df_final == -9).sum()
print("Número de valores -9 por columna:")
print(missing_counts)

Número de valores -9 por columna:
NUMERO            0
CCAA              0
CAPROV            0
TAMAMU            0
DENSIDAD          0
NUMACTI           0
NUMOCU            0
EDADSP            0
REGTEN            0
ZONARES          12
SUPERF            0
AGUACALI          0
CALEF             2
DISPOSIOV         0
NUMOVD            0
IMPEXAC           0
FUENPRINRED       0
GASTOT            0
GASTMON           0
GASTNOM4          0
FACTOR            0
NUMPERI           0
COMITOT           0
NADUL_MAS         0
NADUL_FEM         0
NNINO_MAS         0
NNINO_FEM         0
ANC_MAS           0
ANC_FEM           0
PERCEP_MAS        0
NO_PERCEP_MAS     0
PERCEP_FEM        0
NO_PERCEP_FEM     0
NACION_ESP        0
NACION_EXT        0
EDUC_SUPERIOR     0
Año               0
Tasa_Paro         0
T_med             0
Inflacion         0
Tipo_Interes      0
EUR_USD           0
IPC               0
SC_1              0
SC_2              0
SC_3              0
SC_4              0
SC_5              0
SC_6  

In [54]:
df_final = df_final[(df_final != -9).any(axis=1)]

In [55]:
#Borrar los nan que no sean outputs, los outputs son SC_
df_final = df_final.dropna(subset=[col for col in df_final.columns if not col.startswith('SC_')])


In [56]:
nan_counts = df_final.isna().sum()
nan_counts

NUMERO                0
CCAA                  0
CAPROV                0
TAMAMU                0
DENSIDAD              0
NUMACTI               0
NUMOCU                0
EDADSP                0
REGTEN                0
ZONARES               0
SUPERF                0
AGUACALI              0
CALEF                 0
DISPOSIOV             0
NUMOVD                0
IMPEXAC               0
FUENPRINRED           0
GASTOT                0
GASTMON               0
GASTNOM4              0
FACTOR                0
NUMPERI               0
COMITOT               0
NADUL_MAS             0
NADUL_FEM             0
NNINO_MAS             0
NNINO_FEM             0
ANC_MAS               0
ANC_FEM               0
PERCEP_MAS            0
NO_PERCEP_MAS         0
PERCEP_FEM            0
NO_PERCEP_FEM         0
NACION_ESP            0
NACION_EXT            0
EDUC_SUPERIOR         0
Año                   0
Tasa_Paro             0
T_med                 0
Inflacion             0
Tipo_Interes          0
EUR_USD         

Creamos la variable de no ocupados y no activos

In [57]:
numero_miembros = (df_final["ANC_MAS"].astype(int) + df_final["ANC_FEM"].astype(int) +
    df_final["NADUL_MAS"].astype(int) + df_final["NADUL_FEM"].astype(int) +
    df_final["NNINO_MAS"].astype(int) + df_final["NNINO_FEM"].astype(int))

In [58]:
# Nueva variable no ocupados sera la resta entre ocupados y la suma de ANC_MAS, ANC_FEM, NADUL_MAS, NADUL_FEM, NNINO_MAS, NNINO_FEM
df_final["NUMNOOCU"] = numero_miembros - df_final["NUMOCU"].astype(int)

# Nueva variable no activos sera la resta entre activos y la suma de ANC_MAS, ANC_FEM, NADUL_MAS, NADUL_FEM, NNINO_MAS, NNINO_FEM
df_final["NUMNOACTI"] = numero_miembros -  df_final["NUMACTI"].astype(int)


In [59]:
numocu_counts = df_final["NUMOCU"].value_counts()
numacti_counts = df_final["NUMACTI"].value_counts()

print("Value counts de NUMOCU:")
print(numacti_counts)


Value counts de NUMOCU:
2.0    121891
1.0     92087
0.0     84342
3.0     19361
4.0      4647
5.0       356
6.0        52
7.0         9
8.0         3
Name: NUMACTI, dtype: int64


Gasto no monetario si es Nan es 0

In [60]:
# GASTNOM4 si es NaN, se reemplaza por 0
df_final["GASTNOM4"] = df_final["GASTNOM4"].fillna(0)

Eliminamos la columna de Comunidad Autonoma ya que viene reflejada en la columna CCAA

In [61]:
df_final.drop(columns=['NACION_EXT'], inplace=True) 

In [62]:
df_final.head()  # Mostrar las primeras filas del DataFrame final

,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTOT,GASTMON,GASTNOM4,FACTOR,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,EDUC_SUPERIOR,Año,Tasa_Paro,T_med,Inflacion,Tipo_Interes,EUR_USD,IPC,SC_1,SC_2,SC_3,SC_4,SC_5,SC_6,SC_7,SC_8,SC_9,SC_10,SC_11,SC_12,NUMNOOCU,NUMNOACTI
0,1,9,0,4,3,2.0,2.0,55,5,5,70.0,1,1,0,0.0,5020,autonomYRenta,4.260498e+09,3.669192e+09,591306213.0,9.026143e+08,2.0,58.0,0,0,1,1,0,0,1,0,1,0,1,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,684981607.0,319264308.0,34285154.0,175239084.0,503926195.0,210881486.0,314070831.0,210924601.0,533821695.0,18083827.0,10777868.0,652935187.0,0,0
2,5,9,0,1,1,1.0,1.0,38,5,1,120.0,1,1,0,0.0,1082,asalariado,1.935653e+09,1.532995e+09,402657609.0,8.357985e+08,1.0,72.0,1,1,1,1,0,0,1,1,0,2,1,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,207139885.0,117821010.0,288584808.0,103409729.0,53700469.0,13292166.0,112786301.0,39693246.0,273077146.0,29785347.0,178429999.0,115275171.0,3,3
3,11,9,0,1,2,0.0,0.0,74,6,3,70.0,1,1,0,0.0,1282,pension,2.506370e+09,2.106291e+09,400079199.0,9.048945e+08,2.0,60.0,0,0,0,0,1,1,1,0,1,0,1,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,525744749.0,44211243.0,135197087.0,221094635.0,14721353.0,57875692.0,700587473.0,106605537.0,103096655.0,NaN,22577416.0,174578927.0,2,2
4,34,9,0,4,3,0.0,0.0,48,6,5,56.0,1,1,0,0.0,1685,pension,2.311958e+09,1.361849e+09,950108912.0,1.718298e+09,1.0,23.0,0,0,1,0,0,0,1,0,0,0,1,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,299567157.0,70557541.0,NaN,293301126.0,NaN,NaN,NaN,189679497.0,181678915.0,NaN,300597524.0,26467000.0,1,1
5,64,9,0,1,2,2.0,2.0,47,5,7,120.0,1,1,0,0.0,3000,autonomYRenta,4.038602e+09,3.121573e+09,917028329.0,1.207538e+09,2.0,42.0,1,0,1,1,0,0,1,1,1,0,1,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,740097385.0,202145284.0,20825196.0,576793216.0,114368675.0,5983349.0,205443231.0,67672831.0,463664798.0,31396.0,247682902.0,476865095.0,1,1


### Dividir el GASTOT entre el factor poblacional

In [63]:
# El FACTOR de 2006 a 2015 hay que dividirlo por 1e6
df_final.loc[df_final['Año'].between(2006, 2015), 'FACTOR'] /= 1e6

# El GASTOMON  de 2006 a 2015 hay que dividirlo por 100
df_final.loc[df_final['Año'].between(2006, 2015), 'GASTNOM4'] /= 100

# Todas las supercategorías de 2006 a 2015 hay que dividirlas por 100
for col in df_final.columns:
    if col.startswith("SC_") and df_final['Año'].between(2006, 2015).any():
        df_final.loc[df_final['Año'].between(2006, 2015), col] /= 100


In [64]:
# Genero la lista de columnas a dividir
cols = ['GASTOT', 'GASTMON', 'GASTNOM4'] + [c for c in df_final.columns if c.startswith("SC_")]

# Divido y redondeo de un plumazo
df_final[cols] = df_final[cols].div(df_final['FACTOR'], axis=0).round(2)


### Elevar los ingresos mensuales a ingresos anuales

In [65]:
# Elevar IMPEXAC a Anual

df_final['IMPEXAC'] = df_final['IMPEXAC'] * 12  # Pasar de mensual a anual

In [66]:
# Quiero sacar una columna nueva que sea un porcentaje sacado de IMPEXAC Y GASTMON
df_final['tasaAhorro'] = (
    1 - (df_final['IMPEXAC'] / df_final['GASTMON'])
).fillna(0) * 100  # Multiplicar por 100 para convertir a porcentaje

In [67]:
# Elimina la columna factor
df_final.drop(columns=['FACTOR'], inplace=True)

## Eliminamos la categoria de gasto 42, ya que no refleja ningún gasto monetario

In [68]:
df_final.drop(columns=['GASTOT', "GASTMON"], inplace=True)


## Eliminamos nulos de los inputs

In [69]:
output_columns = [col for col in df_final.columns if col.endswith(tuple(f"_{i}" for i in range(11, 129)))]

non_output_columns = [col for col in df_final.columns if col not in output_columns]
df_final = df_final.dropna(subset=non_output_columns)

### Calculamos la inflación acumulada y la aplicamos a todos las columnas que sean tipo €

In [70]:
inflacion_df = pd.read_csv('../../data/inflacion_espana_2006_2023.csv', sep='\t')

In [71]:
codigos_comunidad = {
    "Andalucía": 1, "Aragón": 2, "Asturias": 3, "Islas Baleares": 4,
    "Canarias": 5, "Cantabria": 6, "Castilla y León": 8, "Castilla-La Mancha": 7,
    "Cataluña": 9, "Comunidad Valenciana": 10, "Extremadura": 11, "Galicia": 12,
    "Madrid": 13, "Murcia": 14, "Navarra": 15,
    "País Vasco": 16, "La Rioja": 17, "Ceuta": 18, "Melilla": 19
}

# Agregar código de comunidad
inflacion_df["CCAA"] = inflacion_df["Comunidad Autónoma"].map(codigos_comunidad)



In [72]:
inflacion_df.head()

,Año,Comunidad Autónoma,Inflación (%),CCAA
0,2006,Andalucía,2.9,1
1,2006,Aragón,2.9,2
2,2006,Asturias,2.4,3
3,2006,Islas Baleares,2.8,4
4,2006,Canarias,2.0,5


In [73]:
def calcular_factores_acumulados_a_2023(df_inflacion):
    """
    Dado un DataFrame con columnas: 'Año', 'Comunidad Autónoma', 'Inflación (%)', 'CCAA',
    devuelve un DataFrame con la inflación acumulada para ajustar precios a 2023.
    """
    df = df_inflacion.copy()
    df = df.sort_values(['CCAA', 'Año'])

    # A porcentaje decimal
    df['inflacion_decimal'] = df['Inflación (%)'] / 100

    # Calcular factor acumulado por CCAA
    df['factor_acumulado'] = df.groupby('CCAA')['inflacion_decimal'].transform(lambda x: (1 + x).cumprod())

    # Obtener el factor en 2023 para cada CCAA
    factores_2023 = df[df['Año'] == 2023].set_index('CCAA')['factor_acumulado'].to_dict()

    # Crear factor de ajuste a 2023: factor_acumulado / factor_2023
    df['factor_ajuste_2023'] = df.apply(
        lambda row: row['factor_acumulado'] / factores_2023.get(row['CCAA'], np.nan),
        axis=1
    )

    return df[['Año', 'Comunidad Autónoma', 'CCAA', 'factor_ajuste_2023']]


In [74]:
inflacion_ajuste = calcular_factores_acumulados_a_2023(inflacion_df)
# Eliminar filas donde Comunidad Autónoma es Nacional
inflacion_ajuste = inflacion_ajuste[inflacion_ajuste['Comunidad Autónoma'] != 'Nacional']

inflacion_ajuste.head()

,Año,Comunidad Autónoma,CCAA,factor_ajuste_2023
0,2006,Andalucía,1,0.722433
19,2007,Andalucía,1,0.752775
38,2008,Andalucía,1,0.762561
57,2009,Andalucía,1,0.767136
76,2010,Andalucía,1,0.791685


In [75]:
def aplicar_ajuste_inflacion(df_final, df_factores):
    """
    Aplica los factores de ajuste a 2023 a las columnas monetarias del df_final,
    modificando directamente las columnas que comienzan por 'SC_' más 'GASTOT' e 'IMPEXAC'.
    """
    df = df_final.copy()

    # Unir el factor a cada fila según Año y CCAA
    df = df.merge(df_factores[['Año', 'CCAA', 'factor_ajuste_2023']],
                  on=['Año', 'CCAA'], how='left')

    # Columnas monetarias a ajustar
    columnas_monetarias = [col for col in df.columns if col.startswith('SC_')]
    columnas_monetarias += [col for col in ['GASTOT', 'GASTMON', 'IMPEXAC', "GASTNOM4"] if col in df.columns]

    # Ajustar directamente las columnas (manteniendo NaNs)
    for col in columnas_monetarias:
        df[col] = df[col] * df['factor_ajuste_2023']

    # Eliminar columna auxiliar
    df.drop(columns=['factor_ajuste_2023'], inplace=True)

    return df


In [76]:
df_final_ajustado = aplicar_ajuste_inflacion(df_final, inflacion_ajuste)

pd.set_option('display.max_columns', None)  # Mostrar todas las columnas en el output
df_final_ajustado.head(5)  # Mostrar las primeras filas del DataFrame ajustado

,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTNOM4,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,EDUC_SUPERIOR,Año,Tasa_Paro,T_med,Inflacion,Tipo_Interes,EUR_USD,IPC,SC_1,SC_2,SC_3,SC_4,SC_5,SC_6,SC_7,SC_8,SC_9,SC_10,SC_11,SC_12,NUMNOOCU,NUMNOACTI,tasaAhorro
0,1,9,0,4,3,2.0,2.0,55,5,5,70.0,1,1,0,0.0,42599.711960,autonomYRenta,4632.676246,2.0,58.0,0,0,1,1,0,0,1,0,1,0,1,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,5366.587817,2501.325816,268.610136,1372.935538,3948.082468,1652.181458,2460.635454,1652.520898,4182.303096,141.680815,84.442756,5115.515411,0,0,98.518107
1,5,9,0,1,1,1.0,1.0,38,5,1,120.0,1,1,0,0.0,9181.850267,asalariado,3406.873777,1.0,72.0,1,1,1,1,0,0,1,1,0,2,1,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,1752.606178,996.878519,2441.704606,874.948865,454.361569,112.467765,954.278807,335.840458,2310.496927,252.012937,1509.687402,975.338226,3,3,99.292104
2,64,9,0,1,2,2.0,2.0,47,5,7,120.0,1,1,0,0.0,25457.995195,autonomYRenta,5370.364086,2.0,42.0,1,0,1,1,0,0,1,1,1,0,1,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,4334.209539,1183.817992,121.957940,3377.858734,669.771567,35.040102,1203.130710,396.310269,2715.342696,0.183863,1450.497563,2792.650141,1,1,98.607389
3,117,9,0,1,2,2.0,2.0,38,6,6,300.0,1,1,0,0.0,71418.162521,asalariado,14052.219328,2.0,104.0,0,2,1,1,0,0,1,0,1,2,1,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,5079.350915,1688.147948,12716.692900,5405.001103,23449.868534,1307.600422,37315.756552,108.146978,8215.005111,936.337992,3427.169457,14719.303662,2,2,99.375543
4,147,9,1,5,3,1.0,1.0,63,6,5,92.0,1,1,0,0.0,30507.164242,pension,5821.055461,2.0,117.0,0,1,2,1,0,0,2,0,0,2,1,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,2938.199157,339.998597,2162.006099,1618.400113,560.783061,2351.986388,10859.263427,1412.982378,10504.548694,0.339440,7082.202113,2041.618068,3,3,99.271424


Renombrar tasaAhorro

In [77]:
# Renombrar tasaAhorro a tasa_ahorro
df_final_ajustado.rename(columns={'tasaAhorro': 'tasa_ahorro'}, inplace=True)

Pasamos el año a timestamp

In [78]:
df_final_ajustado['Timestamp'] = pd.to_datetime(df_final_ajustado['Año'].astype(str) + '-01-01T00:00:00')
df_final_ajustado['Timestamp'] = df_final_ajustado['Timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S')

# Borrar la columna 'Año' si no es necesaria
df_final_ajustado.drop(columns=['Año'], inplace=True)

df_final_ajustado['Timestamp'].head()  

0    2006-01-01T00:00:00
1    2006-01-01T00:00:00
2    2006-01-01T00:00:00
3    2006-01-01T00:00:00
4    2006-01-01T00:00:00
Name: Timestamp, dtype: object

In [79]:
df_final_ajustado.head()  # Mostrar las primeras filas de la columna GASTOT ajustada

,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTNOM4,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,EDUC_SUPERIOR,Tasa_Paro,T_med,Inflacion,Tipo_Interes,EUR_USD,IPC,SC_1,SC_2,SC_3,SC_4,SC_5,SC_6,SC_7,SC_8,SC_9,SC_10,SC_11,SC_12,NUMNOOCU,NUMNOACTI,tasa_ahorro,Timestamp
0,1,9,0,4,3,2.0,2.0,55,5,5,70.0,1,1,0,0.0,42599.711960,autonomYRenta,4632.676246,2.0,58.0,0,0,1,1,0,0,1,0,1,0,1,1,6.5,17.62,2.8,3.5,1.26,78.512833,5366.587817,2501.325816,268.610136,1372.935538,3948.082468,1652.181458,2460.635454,1652.520898,4182.303096,141.680815,84.442756,5115.515411,0,0,98.518107,2006-01-01T00:00:00
1,5,9,0,1,1,1.0,1.0,38,5,1,120.0,1,1,0,0.0,9181.850267,asalariado,3406.873777,1.0,72.0,1,1,1,1,0,0,1,1,0,2,1,0,6.5,17.62,2.8,3.5,1.26,78.512833,1752.606178,996.878519,2441.704606,874.948865,454.361569,112.467765,954.278807,335.840458,2310.496927,252.012937,1509.687402,975.338226,3,3,99.292104,2006-01-01T00:00:00
2,64,9,0,1,2,2.0,2.0,47,5,7,120.0,1,1,0,0.0,25457.995195,autonomYRenta,5370.364086,2.0,42.0,1,0,1,1,0,0,1,1,1,0,1,0,6.5,17.62,2.8,3.5,1.26,78.512833,4334.209539,1183.817992,121.957940,3377.858734,669.771567,35.040102,1203.130710,396.310269,2715.342696,0.183863,1450.497563,2792.650141,1,1,98.607389,2006-01-01T00:00:00
3,117,9,0,1,2,2.0,2.0,38,6,6,300.0,1,1,0,0.0,71418.162521,asalariado,14052.219328,2.0,104.0,0,2,1,1,0,0,1,0,1,2,1,1,6.5,17.62,2.8,3.5,1.26,78.512833,5079.350915,1688.147948,12716.692900,5405.001103,23449.868534,1307.600422,37315.756552,108.146978,8215.005111,936.337992,3427.169457,14719.303662,2,2,99.375543,2006-01-01T00:00:00
4,147,9,1,5,3,1.0,1.0,63,6,5,92.0,1,1,0,0.0,30507.164242,pension,5821.055461,2.0,117.0,0,1,2,1,0,0,2,0,0,2,1,0,6.5,17.62,2.8,3.5,1.26,78.512833,2938.199157,339.998597,2162.006099,1618.400113,560.783061,2351.986388,10859.263427,1412.982378,10504.548694,0.339440,7082.202113,2041.618068,3,3,99.271424,2006-01-01T00:00:00


In [80]:
df_final_ajustado.tail()

,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTNOM4,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,EDUC_SUPERIOR,Tasa_Paro,T_med,Inflacion,Tipo_Interes,EUR_USD,IPC,SC_1,SC_2,SC_3,SC_4,SC_5,SC_6,SC_7,SC_8,SC_9,SC_10,SC_11,SC_12,NUMNOOCU,NUMNOACTI,tasa_ahorro,Timestamp
75814,7086,18,1,4,3,1.0,1.0,62,5,6,72.0,1,0,0,0.0,40800.0,asalariado,6721.75,1.0,14.0,1,0,0,0,0,0,1,0,0,0,1,1,30.0,19.6,3.3,4.5,1.08,112.407167,7311.35,686.61,3406.68,12437.62,1311.52,875.85,1270.67,1205.83,265.79,450.0,1704.43,1054.13,0,0,-27.577862,2023-01-01T00:00:00
75815,7207,18,1,4,3,2.0,1.0,43,1,4,80.0,1,0,0,0.0,11760.0,asalariado,6226.03,1.0,97.0,2,1,1,1,0,0,1,2,0,2,1,0,30.0,19.6,3.3,4.5,1.08,112.407167,25224.53,221.49,6500.95,2164.09,5295.26,2711.39,5735.41,3518.25,6106.47,700.0,5542.20,6255.31,4,3,83.194082,2023-01-01T00:00:00
75816,9330,18,1,4,3,2.0,2.0,58,5,6,90.0,1,0,1,1.0,119664.0,autonomYRenta,8909.07,2.0,6.0,1,1,0,0,0,0,1,0,1,0,1,1,30.0,19.6,3.3,4.5,1.08,112.407167,1972.78,83.06,3069.65,5001.51,488.31,2746.41,7244.67,564.00,3469.99,1200.0,23084.19,15588.99,0,0,-85.486557,2023-01-01T00:00:00
75817,10547,18,1,4,3,3.0,3.0,58,6,5,96.0,1,0,0,0.0,68172.0,asalariado,7148.93,3.0,4.0,1,2,1,0,0,0,1,1,2,0,1,1,30.0,19.6,3.3,4.5,1.08,112.407167,2914.57,17.53,125.23,1854.00,217.51,169.28,574.99,1838.98,388.86,180.0,1500.39,7010.74,1,1,-305.976862,2023-01-01T00:00:00
75818,18153,18,1,4,3,2.0,1.0,46,1,5,90.0,1,0,1,1.0,27120.0,asalariado,10025.32,2.0,34.0,1,1,0,2,0,0,1,0,1,2,1,0,30.0,19.6,3.3,4.5,1.08,112.407167,6566.69,977.16,2945.63,1260.12,2795.53,3146.22,2792.43,960.00,1310.90,15000.0,2971.99,8841.85,3,2,45.287856,2023-01-01T00:00:00


Borramos la columna numero ya que el identificador no va a ser necesario para entrenar nuestro modelo

In [81]:
df_final_ajustado.drop(columns=['NUMERO'], inplace=True)

### Renombramos las columnas de gasto mensual por categorías para que se entienda

In [82]:
def renombrar_columnas_sc(df, categoria_detallada):
    """
    Renombra las columnas SC_{codigo} a una abreviatura legible basada en el nombre de la categoría.
    Ejemplo: SC_011 → ALIMENTOS_011
    """
    df = df.copy()
    columnas_renombradas = {}

    for col in df.columns:
        if col.startswith("SC_"):
            codigo = col.split("_")[1]
            nombre_categoria = categoria_detallada.get(codigo.lstrip("0"), None)

            if nombre_categoria:
                # Tomamos una abreviatura clara: primeras palabras en mayúsculas, separadas por _
                abreviatura = (
                    nombre_categoria.upper()
                    .replace(",", "")
                    .replace(".", "")
                    .replace("/", "_")
                    .replace(" Y ", "_")
                    .replace(" ", "_")
                )
                nuevo_nombre = f"{abreviatura}_{codigo}"
                columnas_renombradas[col] = nuevo_nombre

    return df.rename(columns=columnas_renombradas)

In [83]:
# Diccionario de nombres sin puntos
categoria_detallada = {
        "1": "Alimentos y bebidas no alcohólicas",
        "2": "Bebidas alcohólicas, tabaco y narcótico",
        "3": "Artículos de vestir y calzado",
        "4": "Vivienda, agua, electricidad, gas y otros combustibles",
        "5": "Mobiliario, equipamiento del hogar y gastos de conservación",
        "6": "Salud",
        "7": "Transportes",
        "8": "Comunicaciones",
        "9": "Ocio, espectáculos y cultura ",
        "10": "Enseñanza",
        "11": "Hoteles, cafés y restaurantes",
        "12": "Otros bienes y servicios",
    }

df_final_ajustado = renombrar_columnas_sc(df_final_ajustado, categoria_detallada)

In [84]:

pd.set_option('display.max_columns', None)  # Mostrar todas las columnas en el outpu
 # No Mostrar las filas en el output
df_final_ajustado.head(5)  # Mostrar las primeras filas del DataFrame ajustado

,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTNOM4,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,EDUC_SUPERIOR,Tasa_Paro,T_med,Inflacion,Tipo_Interes,EUR_USD,IPC,ALIMENTOS_BEBIDAS_NO_ALCOHÓLICAS_1,BEBIDAS_ALCOHÓLICAS_TABACO_NARCÓTICO_2,ARTÍCULOS_DE_VESTIR_CALZADO_3,VIVIENDA_AGUA_ELECTRICIDAD_GAS_OTROS_COMBUSTIBLES_4,MOBILIARIO_EQUIPAMIENTO_DEL_HOGAR_GASTOS_DE_CONSERVACIÓN_5,SALUD_6,TRANSPORTES_7,COMUNICACIONES_8,OCIO_ESPECTÁCULOS_CULTURA__9,ENSEÑANZA_10,HOTELES_CAFÉS_RESTAURANTES_11,OTROS_BIENES_SERVICIOS_12,NUMNOOCU,NUMNOACTI,tasa_ahorro,Timestamp
0,9,0,4,3,2.0,2.0,55,5,5,70.0,1,1,0,0.0,42599.711960,autonomYRenta,4632.676246,2.0,58.0,0,0,1,1,0,0,1,0,1,0,1,1,6.5,17.62,2.8,3.5,1.26,78.512833,5366.587817,2501.325816,268.610136,1372.935538,3948.082468,1652.181458,2460.635454,1652.520898,4182.303096,141.680815,84.442756,5115.515411,0,0,98.518107,2006-01-01T00:00:00
1,9,0,1,1,1.0,1.0,38,5,1,120.0,1,1,0,0.0,9181.850267,asalariado,3406.873777,1.0,72.0,1,1,1,1,0,0,1,1,0,2,1,0,6.5,17.62,2.8,3.5,1.26,78.512833,1752.606178,996.878519,2441.704606,874.948865,454.361569,112.467765,954.278807,335.840458,2310.496927,252.012937,1509.687402,975.338226,3,3,99.292104,2006-01-01T00:00:00
2,9,0,1,2,2.0,2.0,47,5,7,120.0,1,1,0,0.0,25457.995195,autonomYRenta,5370.364086,2.0,42.0,1,0,1,1,0,0,1,1,1,0,1,0,6.5,17.62,2.8,3.5,1.26,78.512833,4334.209539,1183.817992,121.957940,3377.858734,669.771567,35.040102,1203.130710,396.310269,2715.342696,0.183863,1450.497563,2792.650141,1,1,98.607389,2006-01-01T00:00:00
3,9,0,1,2,2.0,2.0,38,6,6,300.0,1,1,0,0.0,71418.162521,asalariado,14052.219328,2.0,104.0,0,2,1,1,0,0,1,0,1,2,1,1,6.5,17.62,2.8,3.5,1.26,78.512833,5079.350915,1688.147948,12716.692900,5405.001103,23449.868534,1307.600422,37315.756552,108.146978,8215.005111,936.337992,3427.169457,14719.303662,2,2,99.375543,2006-01-01T00:00:00
4,9,1,5,3,1.0,1.0,63,6,5,92.0,1,1,0,0.0,30507.164242,pension,5821.055461,2.0,117.0,0,1,2,1,0,0,2,0,0,2,1,0,6.5,17.62,2.8,3.5,1.26,78.512833,2938.199157,339.998597,2162.006099,1618.400113,560.783061,2351.986388,10859.263427,1412.982378,10504.548694,0.339440,7082.202113,2041.618068,3,3,99.271424,2006-01-01T00:00:00


Eliminamos NNINOSD, NUMPERI, DISPOSIOV y NHIJOSD ya que esta repetida con las añadidas

In [85]:
df_final_ajustado.drop(columns=['NUMPERI', 'DISPOSIOV'], inplace=True)


### Ordenamos las columnas de nuestro dataframe

In [89]:
columnas_prioritarias = [
    # Variables de identificación y ponderación
    "Timestamp",
    
    "CCAA",        
    "CAPROV",
    "TAMAMU",
    "DENSIDAD",
    "SUPERF",
    "AGUACALI",
    "CALEF",
    "ZONARES",
    "REGTEN",
    "COMITOT",

    # Composición del hogar
    "NUMOCU",
    "NUMNOOCU",
    "NUMACTI",
    "NUMNOACTI",
    "ANC_MAS",
    "ANC_FEM",
    "NADUL_MAS",
    "NADUL_FEM",
    "NNINO_FEM",
    "NNINO_MAS",
    "PERCEP_MAS",
    "PERCEP_FEM",
    "NO_PERCEP_MAS",
    "NO_PERCEP_FEM",

    # Datos del sustentador principal
    "OCUSP",
    "EDADSP",
    "NACION_ESP",
    "EDUC_SUPERIOR",

    # Características de propiedad e ingresos
    "FUENPRINRED",
    "NUMOVD",
    "IMPEXAC",
    "GASTNOM4",
    "tasa_ahorro",
    
    # Variables ambientales y macro
    "T_med",
    "Tasa_Paro",
    "Inflacion",
    "Tipo_Interes",
    "EUR_USD",
    "IPC",

    # --- A partir de aquí, todos los outputs (gasto por categoría) ---
    "ALIMENTOS_BEBIDAS_NO_ALCOHÓLICAS_1",
    "BEBIDAS_ALCOHÓLICAS_TABACO_NARCÓTICO_2",
    "ARTÍCULOS_DE_VESTIR_CALZADO_3",
    "VIVIENDA_AGUA_ELECTRICIDAD_GAS_OTROS_COMBUSTIBLES_4",
    "MOBILIARIO_EQUIPAMIENTO_DEL_HOGAR_GASTOS_DE_CONSERVACIÓN_5",
    "SALUD_6",
    "TRANSPORTES_7",
    "COMUNICACIONES_8",
    "OCIO_ESPECTÁCULOS_CULTURA__9",
    "ENSEÑANZA_10",
    "HOTELES_CAFÉS_RESTAURANTES_11",
    "OTROS_BIENES_SERVICIOS_12"
]


In [90]:
df_final_ajustado.head(5)  # Mostrar las primeras filas del DataFrame ajustado

,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,NUMOVD,IMPEXAC,FUENPRINRED,GASTNOM4,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,EDUC_SUPERIOR,Tasa_Paro,T_med,Inflacion,Tipo_Interes,EUR_USD,IPC,ALIMENTOS_BEBIDAS_NO_ALCOHÓLICAS_1,BEBIDAS_ALCOHÓLICAS_TABACO_NARCÓTICO_2,ARTÍCULOS_DE_VESTIR_CALZADO_3,VIVIENDA_AGUA_ELECTRICIDAD_GAS_OTROS_COMBUSTIBLES_4,MOBILIARIO_EQUIPAMIENTO_DEL_HOGAR_GASTOS_DE_CONSERVACIÓN_5,SALUD_6,TRANSPORTES_7,COMUNICACIONES_8,OCIO_ESPECTÁCULOS_CULTURA__9,ENSEÑANZA_10,HOTELES_CAFÉS_RESTAURANTES_11,OTROS_BIENES_SERVICIOS_12,NUMNOOCU,NUMNOACTI,tasa_ahorro,Timestamp
0,9,0,4,3,2.0,2.0,55,5,5,70.0,1,1,0.0,42599.711960,autonomYRenta,4632.676246,58.0,0,0,1,1,0,0,1,0,1,0,1,1,6.5,17.62,2.8,3.5,1.26,78.512833,5366.587817,2501.325816,268.610136,1372.935538,3948.082468,1652.181458,2460.635454,1652.520898,4182.303096,141.680815,84.442756,5115.515411,0,0,98.518107,2006-01-01T00:00:00
1,9,0,1,1,1.0,1.0,38,5,1,120.0,1,1,0.0,9181.850267,asalariado,3406.873777,72.0,1,1,1,1,0,0,1,1,0,2,1,0,6.5,17.62,2.8,3.5,1.26,78.512833,1752.606178,996.878519,2441.704606,874.948865,454.361569,112.467765,954.278807,335.840458,2310.496927,252.012937,1509.687402,975.338226,3,3,99.292104,2006-01-01T00:00:00
2,9,0,1,2,2.0,2.0,47,5,7,120.0,1,1,0.0,25457.995195,autonomYRenta,5370.364086,42.0,1,0,1,1,0,0,1,1,1,0,1,0,6.5,17.62,2.8,3.5,1.26,78.512833,4334.209539,1183.817992,121.957940,3377.858734,669.771567,35.040102,1203.130710,396.310269,2715.342696,0.183863,1450.497563,2792.650141,1,1,98.607389,2006-01-01T00:00:00
3,9,0,1,2,2.0,2.0,38,6,6,300.0,1,1,0.0,71418.162521,asalariado,14052.219328,104.0,0,2,1,1,0,0,1,0,1,2,1,1,6.5,17.62,2.8,3.5,1.26,78.512833,5079.350915,1688.147948,12716.692900,5405.001103,23449.868534,1307.600422,37315.756552,108.146978,8215.005111,936.337992,3427.169457,14719.303662,2,2,99.375543,2006-01-01T00:00:00
4,9,1,5,3,1.0,1.0,63,6,5,92.0,1,1,0.0,30507.164242,pension,5821.055461,117.0,0,1,2,1,0,0,2,0,0,2,1,0,6.5,17.62,2.8,3.5,1.26,78.512833,2938.199157,339.998597,2162.006099,1618.400113,560.783061,2351.986388,10859.263427,1412.982378,10504.548694,0.339440,7082.202113,2041.618068,3,3,99.271424,2006-01-01T00:00:00


In [91]:
columnas_prioritarias_camel = [
    "timestamp",

    "ccaa",
    "capitalProvincia",
    "tamanoMunicipio",
    "densidad",
    "superficie",
    "aguaCaliente",
    "calefaccion",
    "zonaResidencial",
    "regimenTenencia",
    "comidasTotales",

    "miembros:ocupados",
    "miembros:noOcupados",
    "miembros:activos",
    "miembros:noActivos",
    "miembros:ancianos:masculinos",
    "miembros:ancianos:femeninos",
    "miembros:adultos:masculinos",
    "miembros:adultos:femeninos",
    "miembros:niños:femeninos",
    "miembros:niños:masculinos",
    "miembros:conIngresos:masculinos",
    "miembros:conIngresos:femeninos",
    "miembros:sinIngresos:masculinos",
    "miembros:sinIngresos:femeninos",

    "trabajaSp",
    "edadSp",
    "espanolSp",
    "educacionSuperiorSp",

    "fuentePrincipalIngresos",
    "numeroViviendasAdicionales",
    "ingresosNetos",
    "gastoNoMonetario",
    "tasaAhorro",

    "temperaturaMedia",
    "tasaParo",
    "inflacion",
    "tipoInteres",
    "tasaCambioEurUsd",
    "ipc",

    "gastoMonetario:alimentosYBebidasNoAlcoholicas1",
    "gastoMonetario:bebidasAlcoholicasYTabaco2",
    "gastoMonetario:ropaYCalzado3",
    "gastoMonetario:viviendaAguaElectricidadGasOtrosCombustibles4",
    "gastoMonetario:mobiliarioEquipamientoDelHogarYGastosDeConservacion5",
    "gastoMonetario:salud6",
    "gastoMonetario:transportes7",
    "gastoMonetario:comunicaciones8",
    "gastoMonetario:ocioEspectaculosYCultura9",
    "gastoMonetario:ensenanza10",
    "gastoMonetario:hotelesCafesYRestaurantes11",
    "gastoMonetario:otrosBienesYServicios12"
]


In [92]:
print(len(columnas_prioritarias_camel))  
print(len(columnas_prioritarias))


52
52


In [93]:
def reordenar_columnas(df, columnas_prioritarias):
    # Asegurar que solo usamos las que realmente están en el df
    columnas_prioritarias = [col for col in columnas_prioritarias if col in df.columns]
    
    # Identificar columnas que no están en la lista prioritaria
    columnas_restantes = [col for col in df.columns if col not in columnas_prioritarias]
    
    # Orden final
    orden_final = columnas_prioritarias + columnas_restantes
    
    return df[orden_final]


In [94]:
df = reordenar_columnas(df_final_ajustado, columnas_prioritarias)


In [95]:
df.head(5)  # Mostrar las primeras filas del DataFrame reordenado

,Timestamp,CCAA,CAPROV,TAMAMU,DENSIDAD,SUPERF,AGUACALI,CALEF,ZONARES,REGTEN,COMITOT,NUMOCU,NUMNOOCU,NUMACTI,NUMNOACTI,ANC_MAS,ANC_FEM,NADUL_MAS,NADUL_FEM,NNINO_FEM,NNINO_MAS,PERCEP_MAS,PERCEP_FEM,NO_PERCEP_MAS,NO_PERCEP_FEM,EDADSP,NACION_ESP,EDUC_SUPERIOR,FUENPRINRED,NUMOVD,IMPEXAC,GASTNOM4,tasa_ahorro,T_med,Tasa_Paro,Inflacion,Tipo_Interes,EUR_USD,IPC,ALIMENTOS_BEBIDAS_NO_ALCOHÓLICAS_1,BEBIDAS_ALCOHÓLICAS_TABACO_NARCÓTICO_2,ARTÍCULOS_DE_VESTIR_CALZADO_3,VIVIENDA_AGUA_ELECTRICIDAD_GAS_OTROS_COMBUSTIBLES_4,MOBILIARIO_EQUIPAMIENTO_DEL_HOGAR_GASTOS_DE_CONSERVACIÓN_5,SALUD_6,TRANSPORTES_7,COMUNICACIONES_8,OCIO_ESPECTÁCULOS_CULTURA__9,ENSEÑANZA_10,HOTELES_CAFÉS_RESTAURANTES_11,OTROS_BIENES_SERVICIOS_12
0,2006-01-01T00:00:00,9,0,4,3,70.0,1,1,5,5,58.0,2.0,0,2.0,0,0,0,0,0,1,1,1,1,0,0,55,1,1,autonomYRenta,0.0,42599.711960,4632.676246,98.518107,17.62,6.5,2.8,3.5,1.26,78.512833,5366.587817,2501.325816,268.610136,1372.935538,3948.082468,1652.181458,2460.635454,1652.520898,4182.303096,141.680815,84.442756,5115.515411
1,2006-01-01T00:00:00,9,0,1,1,120.0,1,1,1,5,72.0,1.0,3,1.0,3,0,0,1,1,1,1,1,0,1,2,38,1,0,asalariado,0.0,9181.850267,3406.873777,99.292104,17.62,6.5,2.8,3.5,1.26,78.512833,1752.606178,996.878519,2441.704606,874.948865,454.361569,112.467765,954.278807,335.840458,2310.496927,252.012937,1509.687402,975.338226
2,2006-01-01T00:00:00,9,0,1,2,120.0,1,1,7,5,42.0,2.0,1,2.0,1,0,0,1,0,1,1,1,1,1,0,47,1,0,autonomYRenta,0.0,25457.995195,5370.364086,98.607389,17.62,6.5,2.8,3.5,1.26,78.512833,4334.209539,1183.817992,121.957940,3377.858734,669.771567,35.040102,1203.130710,396.310269,2715.342696,0.183863,1450.497563,2792.650141
3,2006-01-01T00:00:00,9,0,1,2,300.0,1,1,6,6,104.0,2.0,2,2.0,2,0,0,0,2,1,1,1,1,0,2,38,1,1,asalariado,0.0,71418.162521,14052.219328,99.375543,17.62,6.5,2.8,3.5,1.26,78.512833,5079.350915,1688.147948,12716.692900,5405.001103,23449.868534,1307.600422,37315.756552,108.146978,8215.005111,936.337992,3427.169457,14719.303662
4,2006-01-01T00:00:00,9,1,5,3,92.0,1,1,5,6,117.0,1.0,3,1.0,3,0,0,0,1,1,2,2,0,0,2,63,1,0,pension,0.0,30507.164242,5821.055461,99.271424,17.62,6.5,2.8,3.5,1.26,78.512833,2938.199157,339.998597,2162.006099,1618.400113,560.783061,2351.986388,10859.263427,1412.982378,10504.548694,0.339440,7082.202113,2041.618068


In [96]:
# Creamos el diccionario de renombrado
rename_dict = dict(zip(columnas_prioritarias, columnas_prioritarias_camel))

# Aplicarlo al DataFrame (por ejemplo df_final_ajustado)
df_final = df.rename(columns=rename_dict)

In [97]:
rename_dict

{'Timestamp': 'timestamp',
 'CCAA': 'ccaa',
 'CAPROV': 'capitalProvincia',
 'TAMAMU': 'tamanoMunicipio',
 'DENSIDAD': 'densidad',
 'SUPERF': 'superficie',
 'AGUACALI': 'aguaCaliente',
 'CALEF': 'calefaccion',
 'ZONARES': 'zonaResidencial',
 'REGTEN': 'regimenTenencia',
 'COMITOT': 'comidasTotales',
 'NUMOCU': 'miembros:ocupados',
 'NUMNOOCU': 'miembros:noOcupados',
 'NUMACTI': 'miembros:activos',
 'NUMNOACTI': 'miembros:noActivos',
 'ANC_MAS': 'miembros:ancianos:masculinos',
 'ANC_FEM': 'miembros:ancianos:femeninos',
 'NADUL_MAS': 'miembros:adultos:masculinos',
 'NADUL_FEM': 'miembros:adultos:femeninos',
 'NNINO_FEM': 'miembros:niños:femeninos',
 'NNINO_MAS': 'miembros:niños:masculinos',
 'PERCEP_MAS': 'miembros:conIngresos:masculinos',
 'PERCEP_FEM': 'miembros:conIngresos:femeninos',
 'NO_PERCEP_MAS': 'miembros:sinIngresos:masculinos',
 'NO_PERCEP_FEM': 'miembros:sinIngresos:femeninos',
 'OCUSP': 'trabajaSp',
 'EDADSP': 'edadSp',
 'NACION_ESP': 'espanolSp',
 'EDUC_SUPERIOR': 'educacio

Ajustamos nuestro codigos de comunidad a dos digitos

In [98]:
df_final['ccaa'] = df_final['ccaa'].astype(str).str.zfill(2)

In [99]:
df_final.head()  

,timestamp,ccaa,capitalProvincia,tamanoMunicipio,densidad,superficie,aguaCaliente,calefaccion,zonaResidencial,regimenTenencia,comidasTotales,miembros:ocupados,miembros:noOcupados,miembros:activos,miembros:noActivos,miembros:ancianos:masculinos,miembros:ancianos:femeninos,miembros:adultos:masculinos,miembros:adultos:femeninos,miembros:niños:femeninos,miembros:niños:masculinos,miembros:conIngresos:masculinos,miembros:conIngresos:femeninos,miembros:sinIngresos:masculinos,miembros:sinIngresos:femeninos,edadSp,espanolSp,educacionSuperiorSp,fuentePrincipalIngresos,numeroViviendasAdicionales,ingresosNetos,gastoNoMonetario,tasaAhorro,temperaturaMedia,tasaParo,inflacion,tipoInteres,tasaCambioEurUsd,ipc,gastoMonetario:alimentosYBebidasNoAlcoholicas1,gastoMonetario:bebidasAlcoholicasYTabaco2,gastoMonetario:ropaYCalzado3,gastoMonetario:viviendaAguaElectricidadGasOtrosCombustibles4,gastoMonetario:mobiliarioEquipamientoDelHogarYGastosDeConservacion5,gastoMonetario:salud6,gastoMonetario:transportes7,gastoMonetario:comunicaciones8,gastoMonetario:ocioEspectaculosYCultura9,gastoMonetario:ensenanza10,gastoMonetario:hotelesCafesYRestaurantes11,gastoMonetario:otrosBienesYServicios12
0,2006-01-01T00:00:00,09,0,4,3,70.0,1,1,5,5,58.0,2.0,0,2.0,0,0,0,0,0,1,1,1,1,0,0,55,1,1,autonomYRenta,0.0,42599.711960,4632.676246,98.518107,17.62,6.5,2.8,3.5,1.26,78.512833,5366.587817,2501.325816,268.610136,1372.935538,3948.082468,1652.181458,2460.635454,1652.520898,4182.303096,141.680815,84.442756,5115.515411
1,2006-01-01T00:00:00,09,0,1,1,120.0,1,1,1,5,72.0,1.0,3,1.0,3,0,0,1,1,1,1,1,0,1,2,38,1,0,asalariado,0.0,9181.850267,3406.873777,99.292104,17.62,6.5,2.8,3.5,1.26,78.512833,1752.606178,996.878519,2441.704606,874.948865,454.361569,112.467765,954.278807,335.840458,2310.496927,252.012937,1509.687402,975.338226
2,2006-01-01T00:00:00,09,0,1,2,120.0,1,1,7,5,42.0,2.0,1,2.0,1,0,0,1,0,1,1,1,1,1,0,47,1,0,autonomYRenta,0.0,25457.995195,5370.364086,98.607389,17.62,6.5,2.8,3.5,1.26,78.512833,4334.209539,1183.817992,121.957940,3377.858734,669.771567,35.040102,1203.130710,396.310269,2715.342696,0.183863,1450.497563,2792.650141
3,2006-01-01T00:00:00,09,0,1,2,300.0,1,1,6,6,104.0,2.0,2,2.0,2,0,0,0,2,1,1,1,1,0,2,38,1,1,asalariado,0.0,71418.162521,14052.219328,99.375543,17.62,6.5,2.8,3.5,1.26,78.512833,5079.350915,1688.147948,12716.692900,5405.001103,23449.868534,1307.600422,37315.756552,108.146978,8215.005111,936.337992,3427.169457,14719.303662
4,2006-01-01T00:00:00,09,1,5,3,92.0,1,1,5,6,117.0,1.0,3,1.0,3,0,0,0,1,1,2,2,0,0,2,63,1,0,pension,0.0,30507.164242,5821.055461,99.271424,17.62,6.5,2.8,3.5,1.26,78.512833,2938.199157,339.998597,2162.006099,1618.400113,560.783061,2351.986388,10859.263427,1412.982378,10504.548694,0.339440,7082.202113,2041.618068


In [100]:
df_final.tail()

,timestamp,ccaa,capitalProvincia,tamanoMunicipio,densidad,superficie,aguaCaliente,calefaccion,zonaResidencial,regimenTenencia,comidasTotales,miembros:ocupados,miembros:noOcupados,miembros:activos,miembros:noActivos,miembros:ancianos:masculinos,miembros:ancianos:femeninos,miembros:adultos:masculinos,miembros:adultos:femeninos,miembros:niños:femeninos,miembros:niños:masculinos,miembros:conIngresos:masculinos,miembros:conIngresos:femeninos,miembros:sinIngresos:masculinos,miembros:sinIngresos:femeninos,edadSp,espanolSp,educacionSuperiorSp,fuentePrincipalIngresos,numeroViviendasAdicionales,ingresosNetos,gastoNoMonetario,tasaAhorro,temperaturaMedia,tasaParo,inflacion,tipoInteres,tasaCambioEurUsd,ipc,gastoMonetario:alimentosYBebidasNoAlcoholicas1,gastoMonetario:bebidasAlcoholicasYTabaco2,gastoMonetario:ropaYCalzado3,gastoMonetario:viviendaAguaElectricidadGasOtrosCombustibles4,gastoMonetario:mobiliarioEquipamientoDelHogarYGastosDeConservacion5,gastoMonetario:salud6,gastoMonetario:transportes7,gastoMonetario:comunicaciones8,gastoMonetario:ocioEspectaculosYCultura9,gastoMonetario:ensenanza10,gastoMonetario:hotelesCafesYRestaurantes11,gastoMonetario:otrosBienesYServicios12
75814,2023-01-01T00:00:00,18,1,4,3,72.0,1,0,6,5,14.0,1.0,0,1.0,0,0,0,1,0,0,0,1,0,0,0,62,1,1,asalariado,0.0,40800.0,6721.75,-27.577862,19.6,30.0,3.3,4.5,1.08,112.407167,7311.35,686.61,3406.68,12437.62,1311.52,875.85,1270.67,1205.83,265.79,450.0,1704.43,1054.13
75815,2023-01-01T00:00:00,18,1,4,3,80.0,1,0,4,1,97.0,1.0,4,2.0,3,0,0,2,1,1,1,1,0,2,2,43,1,0,asalariado,0.0,11760.0,6226.03,83.194082,19.6,30.0,3.3,4.5,1.08,112.407167,25224.53,221.49,6500.95,2164.09,5295.26,2711.39,5735.41,3518.25,6106.47,700.0,5542.20,6255.31
75816,2023-01-01T00:00:00,18,1,4,3,90.0,1,0,6,5,6.0,2.0,0,2.0,0,0,0,1,1,0,0,1,1,0,0,58,1,1,autonomYRenta,1.0,119664.0,8909.07,-85.486557,19.6,30.0,3.3,4.5,1.08,112.407167,1972.78,83.06,3069.65,5001.51,488.31,2746.41,7244.67,564.00,3469.99,1200.0,23084.19,15588.99
75817,2023-01-01T00:00:00,18,1,4,3,96.0,1,0,5,6,4.0,3.0,1,3.0,1,0,0,1,2,0,1,1,2,1,0,58,1,1,asalariado,0.0,68172.0,7148.93,-305.976862,19.6,30.0,3.3,4.5,1.08,112.407167,2914.57,17.53,125.23,1854.00,217.51,169.28,574.99,1838.98,388.86,180.0,1500.39,7010.74
75818,2023-01-01T00:00:00,18,1,4,3,90.0,1,0,5,1,34.0,1.0,3,2.0,2,0,0,1,1,2,0,1,1,0,2,46,1,0,asalariado,1.0,27120.0,10025.32,45.287856,19.6,30.0,3.3,4.5,1.08,112.407167,6566.69,977.16,2945.63,1260.12,2795.53,3146.22,2792.43,960.00,1310.90,15000.0,2971.99,8841.85


### Datamart para cada comunidad

In [103]:
import os

# Directorio donde van a guardarse los ficheros
output_dir = '../../datamarts/picotaData_supercategory'
os.makedirs(output_dir, exist_ok=True)

for ccaa, group in df_final.groupby('ccaa'):
    df_sub = group.drop(columns=['ccaa'])
    filename = f'hogarCCAA{ccaa}.tsv'
    filepath = os.path.join(output_dir, filename)
    df_sub.to_csv(filepath, sep='\t', index=False)

print("Ficheros guardados en", output_dir)


Ficheros guardados en ../../datamarts/picotaData_supercategory


In [104]:
# Cargar de la ruta de salida un fichero de ejemplo
df_example = pd.read_csv(os.path.join(output_dir, 'hogarCCAA01.tsv'), sep='\t')
df_example.head()

,timestamp,capitalProvincia,tamanoMunicipio,densidad,superficie,aguaCaliente,calefaccion,zonaResidencial,regimenTenencia,comidasTotales,miembros:ocupados,miembros:noOcupados,miembros:activos,miembros:noActivos,miembros:ancianos:masculinos,miembros:ancianos:femeninos,miembros:adultos:masculinos,miembros:adultos:femeninos,miembros:niños:femeninos,miembros:niños:masculinos,miembros:conIngresos:masculinos,miembros:conIngresos:femeninos,miembros:sinIngresos:masculinos,miembros:sinIngresos:femeninos,edadSp,espanolSp,educacionSuperiorSp,fuentePrincipalIngresos,numeroViviendasAdicionales,ingresosNetos,gastoNoMonetario,tasaAhorro,temperaturaMedia,tasaParo,inflacion,tipoInteres,tasaCambioEurUsd,ipc,gastoMonetario:alimentosYBebidasNoAlcoholicas1,gastoMonetario:bebidasAlcoholicasYTabaco2,gastoMonetario:ropaYCalzado3,gastoMonetario:viviendaAguaElectricidadGasOtrosCombustibles4,gastoMonetario:mobiliarioEquipamientoDelHogarYGastosDeConservacion5,gastoMonetario:salud6,gastoMonetario:transportes7,gastoMonetario:comunicaciones8,gastoMonetario:ocioEspectaculosYCultura9,gastoMonetario:ensenanza10,gastoMonetario:hotelesCafesYRestaurantes11,gastoMonetario:otrosBienesYServicios12
0,2006-01-01T00:00:00,0,3,1,90.0,1,0,5,6,68.0,3.0,2,3.0,2,0,0,0,2,1,2,2,1,0,2,47,1,0,asalariado,0.0,25097.315845,2766.339719,97.252121,19.62,12.6,2.9,3.5,1.26,81.54775,1124.040435,265.262879,1442.416574,1005.294153,212.142395,415.723961,2245.277830,909.167244,343.328969,1.502660,514.162656,655.022605
1,2006-01-01T00:00:00,0,2,1,120.0,1,0,5,6,84.0,1.0,2,1.0,2,1,1,0,0,0,1,2,0,0,1,67,1,0,pension,0.0,13073.144143,2470.026677,98.919276,19.62,12.6,2.9,3.5,1.26,81.54775,2581.693117,402.655151,189.927586,518.417785,565.354245,2394.496327,1159.201239,272.465535,1131.170846,0.115589,1866.217334,1014.938631
2,2006-01-01T00:00:00,1,5,2,86.0,1,1,5,5,80.0,0.0,4,0.0,4,0,0,2,0,1,1,1,0,2,1,49,1,0,pension,0.0,18638.766517,3726.539616,99.744862,19.62,12.6,2.9,3.5,1.26,81.54775,3144.359911,376.495859,30410.007268,1144.665891,6227.984897,9623.491350,7574.895853,1521.096732,1221.236545,878.868412,5130.118203,5800.557524
3,2006-01-01T00:00:00,0,1,2,90.0,1,0,1,6,84.0,0.0,3,1.0,2,0,0,0,0,1,2,2,1,0,0,65,1,0,pension,0.0,10403.032475,2307.652679,98.817458,19.62,12.6,2.9,3.5,1.26,81.54775,3527.386563,175.146611,500.906014,687.076949,90.556953,25.928114,1467.810088,444.910246,446.167280,0.007224,771.493223,659.797886
4,2006-01-01T00:00:00,0,1,1,90.0,1,0,1,5,101.0,2.0,2,2.0,2,0,0,0,2,1,1,1,1,0,2,50,1,0,autonomYRenta,0.0,14035.424647,2047.576867,99.459185,19.62,12.6,2.9,3.5,1.26,81.54775,3743.986369,1255.530430,2636.706375,502.993844,4312.671029,282.673510,2878.670797,589.331790,1970.153743,1442.799464,4757.306751,1579.519873
